# Introduction

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

We'll begin by getting a sense of what the data looks like.

In [ ]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Here are the descriptions for some of the columns:

* `name` - The name of the country.
* `area` - The total land and sea area of the country.
* `population` - The country's population.
* `population_growth`- The country's population growth as a percentage.
* `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
* `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
* `area`- The country's total area (both land and water).
* `area_land` - The country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
* `area_water` - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

## Summary Statistics

In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

A few things stick out from the summary statistics in the last screen:

- There's a country with a population of `0`
- There's a country with a population of `7256490011` (or more than 7.2 billion people) 

Let's use subqueries to zoom in on just these countries _without_ using the specific values.

## Exploring Outliers

In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

A few things stick out from the summary statistics in the last screen:

- There's a country with a population of `0`
- There's a country with a population of `7256490011` (or more than 7.2 billion people) 

Let's use subqueries to zoom in on just these countries _without_ using the specific values.

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

## Summary Statistics Revisited

In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

There's a country whose population closes in on 1.4 billion!

## Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [ ]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

## Finding Densely Populated Countries

To finish, we'll build on the query above to find countries that are densely populated.  We'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);